In [1]:
#!/usr/bin/env conda run -n newenv1 python
import sys
print(sys.executable)

In [2]:
import numpy as np
import tabula
import pandas as pd
import PyPDF2
import re
import os
import warnings
import math
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [3]:
# Navigate to http://visualstudio.microsoft.com/downloads. 
# Under the Tools for Visual Studio 2019 tab download the Build Tools for Visual Studio 2019. 
# You’ll then install the tools by checking the C++ build tools option box and clicking Install. 
# do NOT uncheck any of the automatically selected boxes.
# Run the following two commands in cmd mode under the venv you are working it:
# conda install -c conda-forge poppler
# pip install pdftotext
import pdftotext

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [5]:
import h5py

### Inizio routine per dati tipo Testing lab

In [6]:
path = os.getcwd()+'\\DATA'

files=[]
filespath = []

for r,d,f in os.walk(path):
    for file in f:
        if '.pdf' in file:
            files.append(file)
            filespath.append(os.path.join(r,file))

In [7]:
def is_number(s):
    try:
        float(s)
        if math.isnan(float(s)):
            return False
        else: 
            return True
    except:
        return False

In [8]:
# funzione per il regex automatico

def findpattern (df,pattern):
    r=re.compile(pattern)
    for i in np.arange(0,df.shape[0],1):
        for e in np.arange(0,df.shape[1],1):
            if r.match (str(str(df.iloc[i,e]))):
                return df.iloc[i,e],i,e
    

In [9]:
def find_str2(stringtofind, Text, num):
    
    """ find the n° position string after the stringtofind present in a text"""
    
    posit = re.search(stringtofind, Text).start()
    list_str = re.split(' |\r|\n|,|:|;',Text[posit:posit+200])
    try:
        while True:
            list_str.remove('')
    except:
        pass
    return list_str[num]

In [10]:
def find_str(stringtofind, Text):
    
    """ find the string just after the stringtofind present in a text"""
    
    posit = re.search(stringtofind, Text.lower()).start()
    list_str = re.split(' |\r|\n|,|:|;',Text[posit:posit+200])
    try:
        while True:
            list_str.remove('')
    except:
        pass
    return list_str[1]

In [11]:
def new_df_row(pdf_file):
    
    file=files[filespath.index(pdf_file)]
    print(pdf_file)
    
    list_err = []
    
    try:
    
        df = tabula.read_pdf(pdf_file, pages=[1]) # cerca solo nella prima pagina
        
        #    0         1            2           3            4          5       6            7
        #['name','dry_weight','water_cont','void_ratio','degree_sat','UCS','elastic_mod','poisson',
        #   8        9         10      11       12       13       14        15
        #'data8','diameter','force','data11','data12','data13','rock_type','name_file'])

        lista0 = ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-', file]

        for i in np.arange(0,len(df),1):  # 'i' cicla tutte le tabelle
            
            for e in np.arange(0,df[i].shape[0],1):   # 'e' numero riga
                
                
                try:
                
                    #if 'power' in stringa1.lower():
                    #if re.search(r'\bpower\b', stringa1, re.I):     #per cercare power and not horsepower

                    if 'specimen' in str(df[i].iloc[e,0]).lower():         lista0[0] = str(df[i].iloc[0,1])   # nome del campione

                    elif 'dry unit' in str(df[i].iloc[e,0]).lower():         lista0[1] = str(df[i].iloc[e,2]) # Dry unit weight
                    elif 'water content'   in str(df[i].iloc[e,0]).lower():  lista0[2] = str(df[i].iloc[e,2]) # Water content
                    elif 'void ratio' in str(df[i].iloc[e,0]).lower():       lista0[3] = str(df[i].iloc[e,2]) # void ratio                 
                    elif 'saturation' in str(df[i].iloc[e,0]).lower():       lista0[4] = str(df[i].iloc[e,2]) # degree od saturation

                    elif 'unconfined compr' in str(df[i].iloc[e,0]).lower(): 
                        
                        if is_number(df[i].iloc[e+1,1]):                     lista0[5] = str(df[i].iloc[e+1,1]) # UCS 
                        elif is_number(df[i].iloc[e+1,0]):                   lista0[5] = str(df[i].iloc[e+1,0]) # UCS 
                        elif is_number(df[i].iloc[e,1]):                     lista0[5] = str(df[i].iloc[e,1]) # UCS 
                            
  
                    elif 'modulus' in str(df[i].iloc[e,0]).lower():
                        if is_number(df[i].iloc[e,1]):                       lista0[6] = str(df[i].iloc[e,1]) # modulus 
                        elif is_number(df[i].iloc[e+1,0]):                   lista0[6] = str(df[i].iloc[e+1,0]) # modulus                        
        
                    elif 'poisson' in str(df[i].iloc[e,0]).lower():
                
                        if is_number(df[i].iloc[e,1]):                       lista0[7] = str(df[i].iloc[e,1]) # poisson 
                        elif is_number(df[i].iloc[e+1,0]):                   lista0[7] = str(df[i].iloc[e+1,0]) # poisson                
                
                
                
                except:
                    
                    try:
                        
                        if 'dry unit' in str(df[i].iloc[e,0]).lower():  
                            for ee in str(df[i].iloc[e,0]).split(' '):
                                if is_number(ee):     lista0[1] = int(ee) # Dry unit weight  

                        elif 'water content' in str(df[i].iloc[e,0]).lower():  
                            for ee in str(df[i].iloc[e,0]).split(' '):
                                if is_number(ee):     lista0[2] = float(ee) # Water content  

                        elif 'void ratio' in str(df[i].iloc[e,0]).lower():  
                            for ee in str(df[i].iloc[e,0]).split(' '):
                                if is_number(ee):     lista0[3] = float(ee) # void ratio  

                        elif 'saturation' in str(df[i].iloc[e,0]).lower():  
                            for ee in str(df[i].iloc[e,0]).split(' '):
                                if is_number(ee):     lista0[4] = float(ee) # degree od saturation 
                    
                    except Exception as er1:
                        print('Table',i,'row',e,'Error1:',er1)             
                
    except Exception as er1:
        print('Error1:',er1)
        
        
    ## FIND ROCK TYPE
    
    with open(pdf_file, "rb") as f:
        pdf = pdftotext.PDF(f)
        
    Text = pdf[0].lower()   
    
    lista0[14] = find_str(r'description', Text)        
    
    try:
        # find water content

        try:
            str0 = find_str2('water content', Text.lower(), 1)
            if float(str0) >0.01: lista0[2] = str0
        except:
            pass
        try:
            str0 = find_str2('water content', Text.lower(), 2)
            if float(str0) >0.01: lista0[2] = str0 
        except:
            pass
        try:
            str0 = find_str2('water content', Text.lower(), 3)
            if float(str0) >0.01: lista0[2] = str0  
        except:
            pass

    except:
        print('water content string not found')

    try:
        # find dry density
        try:
            str1 = find_str2('unit weight', Text.lower(), 3)
            if float(str1) >1200: lista0[1] = str1
        except:
            pass
        try:
            str01 = find_str2('unit weight', Text.lower(), 2)
            if float(str1) >1200: lista0[1] = str1 
        except:
            pass
        try:
            str1 = find_str2('unit weight', Text.lower(), 1)
            if float(str1) >1200: lista0[1] = str1  
        except:
            pass  
    except:
        print('dry density string not found')
        
        
    try:
        # no.
        try:
            str1 = find_str2('no.', Text.lower(), 4)
            if '-' not in lista0[0] and '-' in str1: lista0[0] = str1
        except:
            pass
        try:
            str1 = find_str2('no.', Text.lower(), 3)
            if '-' not in lista0[0] and  '-' in str1: lista0[0] = str1
        except:
            pass
        try:
            str1 = find_str2('no.', Text.lower(), 2)
            if '-' not in lista0[0] and  '-' in str1: lista0[0] = str1
        except:
            pass  
        
    except:
        print('dry density string not found')
            
    return lista0

In [12]:
df1 = pd.DataFrame(columns=['name','dry_weight','water_cont','void_ratio','degree_sat','UCS','elastic_mod','poisson',
                            'data8','diameter','force','data11','data12','depth','rock_type','name_file'])


for file_path_ in filespath[:]:

    row_ls = new_df_row(file_path_)
    
    df1.loc[len(df1)]=row_ls
    
    
df1    

C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-1_CET.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-2_CET.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-3_CET.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-4_CET.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-5_CET.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\Modulus_BS-6_MAT.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-19_GEOR001658.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-20_GEOR001659.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-21_GEOR001660.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-22_GEOR001661.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-23_GEOR001662.pdf
C:\Users\fpe\JUPYTER\datirocce\1-Bs-Strong & Bs-weak\DATA\UCS_BS-24_GEOR001663.pdf
C:\Users\fpe\JUPYTER\datiroc

,name,dry_weight,water_cont,void_ratio,degree_sat,UCS,elastic_mod,poisson,data8,diameter,force,data11,data12,depth,rock_type,name_file
0,BS-1,2805,0.83,-,-,149.2,52.5,0.26,-,-,-,-,-,-,basalt,Modulus_BS-1_CET.pdf
1,BS-2,2732,0.84,-,-,125.6,54.6,0.26,-,-,-,-,-,-,basalt,Modulus_BS-2_CET.pdf
2,BS-3,2758,0.91,-,-,83.4,58.1,0.14,-,-,-,-,-,-,basalt,Modulus_BS-3_CET.pdf
3,BS - 4,2240,4.78,-,-,24.4,15.9,0.18,-,-,-,-,-,-,basalt,Modulus_BS-4_CET.pdf
4,BS - 5,2578,5.09,-,-,22.6,16.6,0.39,-,-,-,-,-,-,basalt,Modulus_BS-5_CET.pdf
5,BS - 6,2411,2.54,-,-,88.1,34.9,0.24,-,-,-,-,-,-,basalt,Modulus_BS-6_MAT.pdf
6,BS - 19,1819,9.9,-,-,12.36,-,-,-,-,-,-,-,-,amygdaloidal,UCS_BS-19_GEOR001658.pdf
7,BS - 20,1780,10.3,-,-,10.23,-,-,-,-,-,-,-,-,amygdaloidal,UCS_BS-20_GEOR001659.pdf
8,BS - 21,1739,10.0,-,-,8.03,-,-,-,-,-,-,-,-,amygdaloidal,UCS_BS-21_GEOR001660.pdf
9,BS - 22,1798,10.3,-,-,13.68,-,-,-,-,-,-,-,-,amygdaloidal,UCS_BS-22_GEOR001661.pdf


In [13]:
df1.to_hdf('df_testingLab.h5', key='losses')